<a href="https://colab.research.google.com/github/miunprime/colab/blob/main/RDP_ROTG_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#__Google Colab RDP__

In [ ]:
#@title **Create User & Password**

import os

username = "tkimondok" #@param {type:"string"}
password = "123456" #@param {type:"string"}

print("Creating User and Setting it up")

# Creation of user
os.system(f"useradd -m {username}")

# Add user to sudo group
os.system(f"adduser {username} sudo")
    
# Set password of user to 'root'
os.system(f"echo '{username}:{password}' | sudo chpasswd")

# Change default shell from sh to bash
os.system("sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd")

print(f"User created and configured having username `{username}` and password `{password}`")

#@markdown  It takes 4-5 minutes for installation

import os
import subprocess

#@markdown  Visit http://remotedesktop.google.com/headless and copy the command after Authentication

CRP = "" #@param {type:"string"}
Pin = 123456 #@param {type: "integer"}
Autostart = True #@param {type: "boolean"}


class CRD:
    def __init__(self, user):
        os.system("apt update")
        self.installCRD()
        self.installDesktopEnvironment()
        self.installGoogleChorme()
        self.finish(user)
        print("\nRDP created succesfully move to https://remotedesktop.google.com/access")

    @staticmethod
    def installCRD():
        print("Installing Chrome Remote Desktop")
        subprocess.run(['wget', 'https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['dpkg', '--install', 'chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def installDesktopEnvironment():
        print("Installing Desktop Environment")
        os.system("export DEBIAN_FRONTEND=noninteractive")
        os.system("apt install --assume-yes xfce4 desktop-base xfce4-terminal")
        os.system("bash -c 'echo \"exec /etc/X11/Xsession /usr/bin/xfce4-session\" > /etc/chrome-remote-desktop-session'")
        os.system("apt remove --assume-yes gnome-terminal")
        os.system("apt install --assume-yes xscreensaver")
        os.system("systemctl disable lightdm.service")

    @staticmethod
    def installGoogleChorme():
        print("Installing Google Chrome")
        subprocess.run(["wget", "https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(["dpkg", "--install", "google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def finish(user):
        print("Finalizing")
        if Autostart:
            os.makedirs(f"/home/{user}/.config/autostart", exist_ok=True)
            link = "https://colab.research.google.com/github/miunprime/colab/blob/master/Colab%20RDP/Colab%20RDP.ipynb"
            colab_autostart = """[Desktop Entry]
Type=Application
Name=Colab
Exec=sh -c "sensible-browser {}"
Icon=
Comment=Open a predefined notebook at session signin.
X-GNOME-Autostart-enabled=true""".format(link)
            with open(f"/home/{user}/.config/autostart/colab.desktop", "w") as f:
                f.write(colab_autostart)
            os.system(f"chmod +x /home/{user}/.config/autostart/colab.desktop")
            os.system(f"chown {user}:{user} /home/{user}/.config")

        os.system(f"adduser {user} chrome-remote-desktop")
        command = f"{CRP} --pin={Pin}"
        os.system(f"su - {user} -c '{command}'")
        os.system("service chrome-remote-desktop start")
        

        print("Finished Succesfully")


try:
    if CRP == "":
        print("Please enter authcode from the given link")
    elif len(str(Pin)) < 6:
        print("Enter a pin more or equal to 6 digits")
    else:
        CRD(username)
except NameError as e:
    print("'username' variable not found, Create a user first")

#__Google Colab ROTG Tools__

In [ ]:
#@title **Install ROTG tools**
from google.colab import drive
drive.mount('/content/drive')
print("Done! Google Drive Mounted")
from IPython.display import clear_output 
!sudo apt update
!sudo curl -L https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -o /usr/local/bin/yt-dlp
!sudo chmod a+rx /usr/local/bin/yt-dlp
!sudo curl -L https://github.com/tkimondok/ROTG/raw/main/mp4decrypt -o /usr/local/bin/mp4decrypt
!sudo chmod a+rx /usr/local/bin/mp4decrypt
!sudo apt install ffmpeg
!sudo apt install aria2
clear_output()
print("Done! ROTG Tools Installed")

#__Start ROTG Tools__

In [ ]:
#@markdown **Extract Manifest Info**
MPD_URL = '/MYS_DOLBY_index.mpd' #@param {type:"string"}
!yt-dlp --no-warnings --allow-u -F "{MPD_URL}"

In [ ]:
#@markdown <h2>Download Video and Audio</h2>
from IPython.display import clear_output
VIDEO_ID = 'video_avc1_1' #@param {type:"string"}
AUDIO_ID = 'audio_DOLBY_ec-3' #@param {type:"string"}
VIDEO_OUTPUT = 'default.mp4' #@param {type:"string"}
AUDIO_OUTPUT = 'default.m4a' #@param {type:"string"}
VIDEO_DECRYPT_OUTPUT = 'dec.mp4' #@param {type:"string"}
AUDIO_DECRYPT_OUTPUT = 'dec.m4a' #@param {type:"string"}
KID_KEY = 'b73908f4dc0451ccb9b9a5039471fba8:f68fa3ef50900875ee0b4416c1a0b259' #@param {type:"string"}
FINAL_OUTPUT = 'done.mp4' #@param {type:"string"}
print("Downloading Encrypted video.....")
!yt-dlp --no-warnings --allow-u -f "{VIDEO_ID}" "{MPD_URL}" -o "{VIDEO_OUTPUT}" -N 60
clear_output()
print("Success")
clear_output()
print("Downloading Encrypted audio.....")
!yt-dlp --no-warnings --allow-u -f "{AUDIO_ID}" "{MPD_URL}" -o "{AUDIO_OUTPUT}" -N 60
clear_output()
print("Success")
clear_output()
print("Decrypting video.....")
!mp4decrypt --show-progress --key "{KID_KEY}" "{VIDEO_OUTPUT}" "{VIDEO_DECRYPT_OUTPUT}"
clear_output()
print("Success")
clear_output()
print("Decrypting audio.....")
!mp4decrypt --show-progress --key "{KID_KEY}" "{AUDIO_OUTPUT}" "{AUDIO_DECRYPT_OUTPUT}"
clear_output()
print("Success")
clear_output()
print("Merging video and audio into single file.....")
!ffmpeg -loglevel error -i "{VIDEO_DECRYPT_OUTPUT}" -i "{AUDIO_DECRYPT_OUTPUT}" -c copy "{FINAL_OUTPUT}"
clear_output()
print("Success")
clear_output()
print("Done! Video Ready to Download.")

#__Download Subtitle from MPD__

In [ ]:
#@title **Get List And Download**
print("Get List....")
!yt-dlp --no-warnings --allow-u --list-subs "{MPD_URL}"
print("Downloading....")
!yt-dlp --no-warnings --allow-u -N 32 --skip-download --all-subs "{MPD_URL}" -o subtitle
clear_output()
print("Done! All Subtitle Downloaded")

In [ ]:
#@title **Convert VTT to SRT**
SUBTITLE_INPUT_01 = 'subtitle.MA.vtt' #@param {type:"string"}
SUBTITLE_OUTPUT_01 = 'MA.srt' #@param {type:"string"}
!ffmpeg -loglevel error -i "{SUBTITLE_INPUT_01}" "{SUBTITLE_OUTPUT_01}"
SUBTITLE_INPUT_02 = 'subtitle.EN.vtt' #@param {type:"string"}
SUBTITLE_OUTPUT_02 = 'EN.srt' #@param {type:"string"}
!ffmpeg -loglevel error -i "{SUBTITLE_INPUT_02}" "{SUBTITLE_OUTPUT_02}"
SUBTITLE_INPUT_03 = 'subtitle.CN.vtt' #@param {type:"string"}
SUBTITLE_OUTPUT_03 = 'CN.srt' #@param {type:"string"}
!ffmpeg -loglevel error -i "{SUBTITLE_INPUT_03}" "{SUBTITLE_OUTPUT_03}"
clear_output()
print("Done! All Subtitle Converted to SRT")

#__Mux Video and Subtitle__

In [ ]:
#@markdown **Mux Video and Subtitle**
SUBTITLE_01 = 'EN.srt' #@param {type:"string"}
SUBTITLE_02 = 'MA.srt' #@param {type:"string"}
SUBTITLE_03 = 'CN.srt' #@param {type:"string"}
LANGUAGE_01 = 'english' #@param {type:"string"}
LANGUAGE_02 = 'malaysia' #@param {type:"string"}
LANGUAGE_03 = 'china' #@param {type:"string"}
MUX_OUTPUT = 'ROTG.Tools.1080p.WEB-DL.DDP5.1.Atmos.x265.HEVC-AAC.H264-ROTG.mkv' #@param {type:"string"}
print("Processing.....")
!ffmpeg -i "{FINAL_OUTPUT}" -i "{SUBTITLE_01}" -i "{SUBTITLE_02}" -i "{SUBTITLE_03}" -map 0 -map 1:s -map 2:s -map 3:s -c copy -metadata:s:s:0 language="{LANGUAGE_01}" -metadata:s:s:1 language="{LANGUAGE_02}" -metadata:s:s:2 language="{LANGUAGE_03}" "{MUX_OUTPUT}"
clear_output()
print("Done! Video and Subtitle Muxed")

#__Copy Muxed Subtitle to Google Drive__

In [ ]:
#@markdown **Copy From Root to Destination**
DESTINATION_MIUN = '' #@param {type:"string"}
DESTINATION_ROTG = '' #@param {type:"string"}
DESTINATION_IMAN = '' #@param {type:"string"}
DESTINATION_XXXX = '' #@param {type:"string"}
print("Processing.....")
!cp -r "{MUX_OUTPUT}" "{DESTINATION_MIUN}"
!cp -r "{MUX_OUTPUT}" "{DESTINATION_ROTG}"
!cp -r "{MUX_OUTPUT}" "{DESTINATION_IMAN}"
!cp -r "{MUX_OUTPUT}" "{DESTINATION_XXXX}"
clear_output()
print("Done! Video Moved to Google Drive")

#__Remove All File In Root__

In [ ]:
#@markdown **Start Remove**
!rm -r "{VIDEO_OUTPUT}" 
!rm -r "{AUDIO_OUTPUT}" 
!rm -r "{VIDEO_DECRYPT_OUTPUT}"
!rm -r "{AUDIO_DECRYPT_OUTPUT}"
!rm -r "{FINAL_OUTPUT}" 
!rm -r "{SUBTITLE_INPUT_01}"
!rm -r "{SUBTITLE_OUTPUT_01}" 
!rm -r "{SUBTITLE_INPUT_02}"
!rm -r "{SUBTITLE_OUTPUT_02}" 
!rm -r "{SUBTITLE_INPUT_03}"
!rm -r "{SUBTITLE_OUTPUT_03}"
!rm -r "{MUX_OUTPUT}"
print("Done! All File in Root Removed")

# **Google Drive Tools**

In [ ]:
#@title **Copy SubFolder Shareddrive**
!ls '/content/drive/Shareddrives'
SHAREDDRIVES = '' #@param {type:"string"}
DESTINATIONS = '' #@param {type:"string"}
print("Processing.....")
!cp -av "{SHAREDDRIVES}" "{DESTINATIONS}"
clear_output()
print("Done!")

In [ ]:
#@title **Copy File**
SOURCE_FILE1 = '' #@param {type:"string"}
SOURCE_FILE2 = '' #@param {type:"string"}
SOURCE_FILE3 = '' #@param {type:"string"}
SOURCE_FILE4 = '' #@param {type:"string"}
SOURCE_FILE5 = '' #@param {type:"string"}
DESTINATIONS = '' #@param {type:"string"}
print("Processing.....")
!cp -r "{SOURCE_FILE1}" "{DESTINATIONS}"
clear_output()
!cp -r "{SOURCE_FILE2}" "{DESTINATIONS}"
clear_output()
!cp -r "{SOURCE_FILE3}" "{DESTINATIONS}"
clear_output()
!cp -r "{SOURCE_FILE4}" "{DESTINATIONS}"
clear_output()
!cp -r "{SOURCE_FILE5}" "{DESTINATIONS}"
clear_output()
print("Done!")

In [ ]:
#@title **Move File**
SOURCE_FILE1 = '' #@param {type:"string"}
SOURCE_FILE2 = '' #@param {type:"string"}
SOURCE_FILE3 = '' #@param {type:"string"}
SOURCE_FILE4 = '' #@param {type:"string"}
SOURCE_FILE5 = '' #@param {type:"string"}
DESTINATIONS = '' #@param {type:"string"}
print("Processing.....")
!mv "{SOURCE_FILE1}" "{DESTINATIONS}"
clear_output()
!mv "{SOURCE_FILE2}" "{DESTINATIONS}"
clear_output()
!mv "{SOURCE_FILE3}" "{DESTINATIONS}"
clear_output()
!mv "{SOURCE_FILE4}" "{DESTINATIONS}"
clear_output()
!mv "{SOURCE_FILE5}" "{DESTINATIONS}"
clear_output()
print("Done!")

In [ ]:
#@title **Download File to Google Drive**
SOURCE_URI1 = "" #@param {type:"string"}
SOURCE_URI2 = "" #@param {type:"string"}
SOURCE_URI3 = "" #@param {type:"string"}
SOURCE_URI4 = "" #@param {type:"string"}
SOURCE_URI5 = "" #@param {type:"string"}
DESTINATION = "" #@param {type:"string"}
print("Downloading.....")
!aria2c --file-allocation=none -x16 "{SOURCE_URI1}" --dir="{DESTINATION}"
!aria2c --file-allocation=none -x16 "{SOURCE_URI2}" --dir="{DESTINATION}"
!aria2c --file-allocation=none -x16 "{SOURCE_URI3}" --dir="{DESTINATION}"
!aria2c --file-allocation=none -x16 "{SOURCE_URI4}" --dir="{DESTINATION}"
!aria2c --file-allocation=none -x16 "{SOURCE_URI5}" --dir="{DESTINATION}"
clear_output()
print("File Downloaded to Google Drive")

In [ ]:
#@title **System Detail**
!free -h
!nvidia-smi -L
!lscpu |grep 'Model name'
!lscpu | grep 'Core(s) per socket:'
!lscpu | grep 'Thread(s) per core'
!nvidia-smi
!nvcc --version